# Delegation-based managers revisited

In [2]:
class Person:
    def __init__(self, name, job=None, pay=0):
        self.name = name
        self.job = job
        self.pay = pay
    def lastName(self):
        return self.name.split()[-1]
    def giveRaise(self, percent):
        self.pay = int(self.pay * (1 + percent))
    def __repr__(self):
        return '[Person: %s, %s]' % (self.name, self.pay)

class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, "mgr", pay)  # Embed a Person object
    def giveRaise(self, percent, bonus=.10):
        self.person.giveRaise(percent + bonus)  # Intercept and delegate
    def __getattr__(self, attr):
        return getattr(self.person, attr)       # Delegate all other attrs
    def __repr__(self):
        return str(self.person)             # Must overload again (in 3.X)

sue = Person("Sue Jones", job='dev', pay=100000)
print(sue. lastName())
sue.giveRaise(.10)
print(sue)
tom = Manager("Tom Jones", 50000)       # Manager.__init__
print(tom.lastName())                   # Manager.__getattr__ -> Person.lastName
tom.giveRaise(.10)                      # Manager.giveRaise -> Person.giveRaise
print(tom)                              # Manager.__repr__ -> Person.__repr__

Jones
[Person: Sue Jones, 110000]
Jones
[Person: Tom Jones, 60000]


The wrapper class's `__repr__` is invoked, and it delegates to the embedded **Person** object's `__repr__`.

Let's *delete* the `Manager.__repr__` method, then Python does not route its attribute fetch through the generic `__getattr__` interceptor. Instead, a default `__repr__` display method inherited from the class's implicit **object** superclass is looked up and run.

In [3]:
# Delete the Manager `__str__` method

class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, 'mgr', pay)      # Embed a Person object
    def giveRaise(self, percent, bonus=.10):
        self.person.giveRaise(percent + bonus)      # Intercept and delegate
    def __getattr__(self, attr):
        return getattr(self.person, attr)           # Delegate all other attrs
    
sue = Person("Sue Jones", job='dev', pay=100000)
print(sue. lastName())
sue.giveRaise(.10)
print(sue)
tom = Manager("Tom Jones", 50000)       # Manager.__init__
print(tom.lastName())                   # Manager.__getattr__ -> Person.lastName
tom.giveRaise(.10)                      # Manager.giveRaise -> Person.giveRaise
print(tom)                              # Manager.__repr__ -> Person.__repr__

Jones
[Person: Sue Jones, 110000]
Jones


### Switch to `__getattribute__`

In [4]:
# Replace `__getattr__` with `__getattribute__`

class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, 'mgr', pay)          # Embed a person object
    def giveRaise(self, percent, bonus=.10):            # Intercept and delegate
        self.person.giveRaise(percent+bonus)
    def __getattribute__(self, attr):
        print("**", attr)
        if attr in ['person', 'giveRaise']:
            return object.__getattribute__(self, attr)      # Fetch my attrs
        else:
            return getattr(self.person, attr)               # Delefate all others

   
sue = Person("Sue Jones", job='dev', pay=100000)
print(sue.lastName())
sue.giveRaise(.10)
print(sue)
tom = Manager("Tom Jones", 50000)       
print(tom.lastName())
tom.giveRaise(.10)
print(tom)        

Jones
[Person: Sue Jones, 110000]
** lastName
** person
Jones
** giveRaise
** person


The `__getattribute__` gets called twice here for methods--once for the method name, and again for the **self.person** embedded object fetch. We could avoid that with a different coding, but we would have to redefine `__repr__` to catch printing

In [5]:
# Code __getattribute__ differently to minimize extra calls

class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, 'mgr', pay)
    def __getattribute__(self, name):
        print("**", name)
        person = object.__getattribute__(self, 'person')
        if name == 'giveRaise':
            return lambda percent: person.giveRaise(percent+.10)
        else:
            return getattr(person, name)
    def __repr__(self):
        person = object.__getattribute__(self, 'person')
        return str(person)
    
sue = Person("Sue Jones", job='dev', pay=100000)
print(sue.lastName())
sue.giveRaise(.10)
print(sue)
tom = Manager("Tom Jones", 50000)       
print(tom.lastName())
tom.giveRaise(.10)
print(tom)      

Jones
[Person: Sue Jones, 110000]
** lastName
Jones
** giveRaise
[Person: Tom Jones, 60000]


When this alternative runs, our object prints properly, but only because we've added an explicit `__repr__` in the wrapper--this attribute is still not routed to our generic attribute interception method.

Delegation-based classes like **Manager** must redefine some operator overloading methods (like `__repr__` and `__str__`) to route them to embedded object in Python 3.X.